In [1]:
import enum
import hashlib
import re
from typing import Any

from pydantic import (
    BaseModel,
    EmailStr,
    Field,
    field_validator,
    model_validator,
    SecretStr,
    ValidationError,
)

In [ ]:
# regex pra validar a senha e o nome
VALID_PASSWORD_REGEX = re.compile(r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d).{8,}$")
VALID_NAME_REGEX = re.compile(r"^[a-zA-Z]{2,}$")

In [3]:
class Role(enum.IntFlag):
    Author = 1
    Editor = 2
    Admin = 4
    SuperAdmin = 8

In [ ]:
class User(BaseModel):
    # field: nome
    name: str = Field(examples=["Arjan"])

    # field: email
    email: EmailStr = Field(
        examples=["user@arjancodes.com"],
        description="The email address of the user",
        frozen=True,
    )

    # field: senha
    password: SecretStr = Field(
        examples=["Password123"], description="The password of the user"
    )

    # field: cargo
    role: Role = Field(
        default=None, description="The role of the user", examples=[1, 2, 4, 8]
    )

    # =-=-=-=-=-=- NAME VALIDATOR =-=-=-=-=-=
    @field_validator("name")
    @classmethod
    def validate_name(cls, v: str) -> str: # pega o nome e o retorna
        if not VALID_NAME_REGEX.match(v): # passa pelo filtro do regex
            raise ValueError( # se erro, retorna mensagem
                "Name is invalid, must contain only letters and be at least 2 characters long"
            )
        return v # retorna o nome validado

    # =-=-=-=-=-=- FIELD VALIDATOR =-=-=-=-=-=
    @field_validator("role", mode="before")
    @classmethod
    def validate_role(cls, v: int | str | Role) -> Role:
        # dicionário que mapeia tipos para funções de conversão
        op = {
            int: lambda x: Role(x),      # se for int, converte para role usando o valor
            str: lambda x: Role[x],      # se for string, busca o role pelo nome
            Role: lambda x: x            # se já for role, retorna como está
        }

        try:
            return op[type(v)](v) # pega o tipo do valor recebido e aplica a função correspondente
        except (KeyError, ValueError):
            raise ValueError(
                f"Role is invalid, please use one of the following: {', '.join([x.name for x in Role])}"
            )

    # =-=-=-=-=-=- MODEL VALIDATOR =-=-=-=-=-=
    @model_validator(mode="before")
    @classmethod
    def validate_user(cls, v: dict[str, Any]) -> dict[str, Any]:
        # verifica se os campos obrigatórios estão presentes
        if "name" not in v or "password" not in v:
            raise ValueError("Name and password are required")

        # verifica se o nome não está contido na senha (case-insensitive)
        if v["name"].casefold() in v["password"].casefold():
            raise ValueError("Password cannot contain name")

        # valida a senha usando regex
        if not VALID_PASSWORD_REGEX.match(v["password"]):
            raise ValueError(
                "Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number"
            )

        # hash da senha usando SHA-256 antes de armazenar
        v["password"] = hashlib.sha256(v["password"].encode()).hexdigest()
        return v

In [5]:
def validate(data: dict[str, Any]) -> None:
    try:
        user = User.model_validate(data)
        print(user)
    except ValidationError as e:
        print("User is invalid:")
        print(e)

In [6]:
test_data = dict(
        good_data={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Admin",
        },
        bad_role={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Password123",
            "role": "Programmer",
        },
        bad_data={
            "name": "Arjan",
            "email": "bad email",
            "password": "bad password",
        },
        bad_name={
            "name": "Arjan<-_->",
            "email": "example@arjancodes.com",
            "password": "Password123",
        },
        duplicate={
            "name": "Arjan",
            "email": "example@arjancodes.com",
            "password": "Arjan123",
        },
        missing_data={
            "email": "<bad data>",
            "password": "<bad data>",
        },
    )

for example_name, data in test_data.items():
        print(example_name)
        validate(data)
        print()

good_data
name='Arjan' email='example@arjancodes.com' password=SecretStr('**********') role=<Role.Admin: 4>

bad_role
User is invalid:
1 validation error for User
role
  Value error, Role is invalid, please use one of the following: Author, Editor, Admin, SuperAdmin [type=value_error, input_value='Programmer', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_data
User is invalid:
1 validation error for User
  Value error, Password is invalid, must contain 8 characters, 1 uppercase, 1 lowercase, 1 number [type=value_error, input_value={'name': 'Arjan', 'email'...ssword': 'bad password'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

bad_name
User is invalid:
1 validation error for User
name
  Value error, Name is invalid, must contain only letters and be at least 2 characters long [type=value_error, input_value='Arjan<-_->', input_type=str]
    For further information visit https://